In [3]:
# 数据分析/处理
import numpy as np
import pandas as pd

# 搭建神经网络
import torch
import torch.nn as nn

# 数据可视化
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# 宇宙的答案
np.random.seed(42)
torch.manual_seed(42)

### 载入数据，分析同task1

In [ ]:
test=pd.read_csv("task1\\test.tsv")
train=pd.read_csv("task1\\train.tsv")
